## <center>Deep Learning and Neural Networks: Final Project</center>
### This notebook contains the testing and inferencing of the test set on the mentioned models:
- The finetuned models include <b>Pegasus-Large, BART-Base, T5-Base</b> [Fine Tuning shown in other notebook].
- Zero-shot predictions using <b>GPT-3 (text-davinci-003)</b> API (provided by OpenAI)
- Zero-shot predictions using <b>GPT-4 (ChatGPT)</b>

## Installing necessary modules

In [ ]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
# Installing the BLURR library and Bert-Score package

!pip install ohmeow-blurr -q
!pip install bert-score -q
!pip install wandb
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 887.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling to

## Importing the required modules

In [ ]:
import pandas as pd
from fastai.text.all import *
from transformers import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *
from fastai.callback.wandb import *

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/blurr/text/modeling/question_answering.py:31:

## Importing the Test Set from PubMed-Summarization Dataset

In [ ]:
# Load the dataset from Hugging Face

from datasets import load_dataset
dataset = load_dataset('ccdv/pubmed-summarization', split="test[:100]")
test_data = pd.DataFrame(dataset)
# Preprocess and taking first 1000 samples.

test_data['article'] = test_data['article'].apply(lambda x: x.replace('\n',''))
test_data = test_data.head(20)
test_data.shape

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pubmed-summarization downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b. Subsequent calls will reuse this data.


(20, 2)

## OpenAI API calls for GPT-3

In [3]:
# installing openai package

!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.8 MB/s eta 0:00:00


In [1]:
# importing warnings package to ignore the depreciation warnings

import warnings
warnings.filterwarnings("ignore")

In [3]:
import openai
import time


openai.api_key = "PUT YOUR API KEY HERE"


# Functiont to fetch summary using OpenAI's API
def get_GPT_summary(text, summary_wc, model, toks):

    # Prompt prefix, this is how the prompt start
    prompt_prefix = "Professionally summarize this medical article like a doctor with about"
    summary_wc = min(summary_wc - (summary_wc % 10), 250)

    try: 
      start_time = time.time()
      response = openai.Completion.create(
      model=model,
      prompt=f"{prompt_prefix} {summary_wc} to {summary_wc+50} words :\n{text}",
      temperature=0.4,
      max_tokens=toks
      )
      end_time = time.time()
      return response['choices'][0]['text'], round(end_time - start_time, 6), response['usage']['total_tokens']
    except: 
      pass
   


# Initializing the parameters for making API call
model = 'text-davinci-003'
summary_word_count = 1000
tokens = 256
results_df = pd.DataFrame(columns=['index', 'article', 'GPT_summary', 'reference_summary'])

for index, row in test_data.iterrows():
   
   output = get_GPT_summary(row.article, len(row.abstract.split()), model, tokens)
   
   if output: 
     outcome, inf_time, tokens_used = output
     # write all data for this record to a dataframe
     res = {'index': index, 
          'article': row.article,
          'GPT_summary': outcome, 
          'reference_summary': row.abstract
          }
     results_df = results_df.append(res, ignore_index=True)


In [ ]:
# Results obtained form GPT-3 API Call
results_df

,index,article,GPT_summary,reference_summary
0,0,"anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . overall , neuropsych...","\n\nAnxiety affects the quality of life of those living with Parkinson's Disease (PD) more than overall cognitive status, motor deficits, apathy, and depression. Although anxiety and depression are often related and coexist in PD patients, research suggests that anxiety is the most prominent and prevalent mood disorder. Current understanding of anxiety and its impact on cognition in PD, as well as its neural basis and best treatment practices, is limited. Neuropsychiatric symptoms in PD have been associated with poorer cognitive performance. A study of 27 PD patients showed a negative rel...","research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and ..."
1,1,"small non - coding rnas are transcribed into mrna but remain untranslated in eukaryotic cells . they include sirna ( small interfering rna ) , mirna ( microrna ) , pirna ( piwi - interacting rna ) and snorna ( small nucleolar rna ) . mirnas are a class of multifunctional singled - stranded small rna which are ~20 nt in length and regulate the stability or translational efficiency of targeted messenger rna depending on the base - pairing complementarity between the mirna and its target mrna [ 1 , 2 ] . although over 1,000 mirna sequences have been identified from the tissues or cells of ...","\n\nSmall non-coding RNAs, including siRNA, miRNA, piRNA, and snoRNA, are transcribed but remain untranslated in eukaryotic cells. miRNAs are a class of multifunctional single-stranded small RNAs that regulate the stability and translational efficiency of targeted mRNAs. Aberrant expression of miRNAs has been observed in breast cancer and is associated with tumorigenesis, metastasis, and apoptosis. miRNAs regulate the cell cycle, cell growth, epithelial-to-mesenchymal transition, and cancer stem cell characteristics. miRNAs may also serve as novel diagnostic and prognostic biomarkers for b...","small non - coding rnas include sirna , mirna , pirna and snorna . \n the involvement of mirnas in the regulation of mammary gland tumorigenesis has been widely studied while the role for other small non - coding rnas remains unclear . here \n we summarize the involvement of mirna in breast cancer onset and progression through regulating the cell cycle and cellular proliferation . \n the regulation of breast cancer stem cells and tumor regeneration by mirna is reviewed . \n in addition , the emerging evidence demonstrating the involvement of pirna and snorna in breast cancer is briefly des..."
2,3,"congenital adrenal hyperplasia ( cah ) refers to a group of autosomal recessive disorders caused by an enzyme deficiency which leads to defects in biosynthesis of steroid precursors . depending on the severity and degree of 21 hydroxylase deficiency , the clinical spectrum may vary from mild form of non classical cah to classic cah . h

**Generated Summary**

In [ ]:
results_df["GPT_summary"][0]

" \n\nAnxiety affects the quality of life of those living with Parkinson's Disease (PD) more than overall cognitive status, motor deficits, apathy, and depression. Although anxiety and depression are often related and coexist in PD patients, research suggests that anxiety is the most prominent and prevalent mood disorder. Current understanding of anxiety and its impact on cognition in PD, as well as its neural basis and best treatment practices, is limited. Neuropsychiatric symptoms in PD have been associated with poorer cognitive performance. A study of 27 PD patients showed a negative relationship between anxiety and cognitive performance. Two further studies examined the influence of symptom laterality on anxiety and cognition, but their findings were inconsistent. This study compared the cognition of PD patients with and without anxiety, excluding depression. Results showed that PD patients with anxiety had worse performance on attentional set-shifting, working memory, and verbal m

**Ground Truth Summary**

In [ ]:
results_df["reference_summary"][0]

"research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . \n group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . \n results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail making task

## Testing the Fine-Tuned Models

In [ ]:
# Mounting Google Drive, so that .pkl file can be imported to colab
from google.colab import drive
drive.mount('/content/drive')

### Evaluation

<b>Loading BART-BASE Model</b>

In [ ]:
bart_base_model = load_learner(fname="/content/drive/MyDrive/Summarization_Models/bart_base_summary_export.pkl")

<b>Loading T5-BASE Model</b>

In [ ]:
t5_base_model = load_learner(fname="/content/drive/MyDrive/Summarization_Models/t5_base_summary_export.pkl")

<b>Loading Pegasus-Large Model</b>

In [ ]:
pegasus_large_model = load_learner(fname="/content/drive/MyDrive/Summarization_Models/pegasus_summary_export.pkl")

<b>Generating Summaries</b>

In [ ]:
# Initializing this dataframe, which stores the summaries for all the models
results_BartT5_Pegasus_df = pd.DataFrame(columns=["article", "Bart_Gen_Summary", "T5_Gen_Summary", "Pegasus_Gen_Summary", "reference_summary"])

# Iterating over the articles to be summarized
for _, row in results_df.iterrows():

  # Inferencing using BART
  bart_res = bart_base_model.blurr_summarize(row.article)

  # Inferencing using T5
  t5_res = t5_base_model.blurr_summarize(row.article)

  # Inferencing using Pegasus
  pegasus_res = pegasus_large_model.blurr_summarize(row.article)

  # Creating a dictionary of results obtained
  gen_res = {
      "article": row.article, 
      "Bart_Gen_Summary": bart_res[0]['summary_texts'], 
      "T5_Gen_Summary": t5_res[0]['summary_texts'], 
      "Pegasus_Gen_Summary": pegasus_res[0]['summary_texts'],
      "reference_summary": row.reference_summary
  }

  # Appending results to the DataFrame
  results_BartT5_Pegasus_df = results_BartT5_Pegasus_df.append(gen_res, ignore_index = True)

In [ ]:
# Checking out results form the Models
results_BartT5_Pegasus_df

,article,Bart_Gen_Summary,T5_Gen_Summary,Pegasus_Gen_Summary,reference_summary
0,"anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet , our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . overall , neuropsych...","background : anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression . \n however , few studies have specifically investigated the relationship between anxiety and cognition in pd and more specifically how anxiety might influence particular domains of cognition such as attention and memory and executive functioning.materials and methods : seventeen patients with anxiety and thirty - three pd patients without anxiety were included in this study.results : anxiety was significantly assoc...","background : recent studies have demonstrated that anxiety affects cognition in patients with parkinson 's disease ( pd ) more than overall cognitive status , motor deficits , and depression . however , the correlation between anxiety and cognition has not been clearly defined.objectives:the aim of this study was to investigate the relationship between anxiety - induced cognitive impairment ( cvs. ) and cognitive function.patients and methods : a total of the repeatable battery for the assessment of neuropsychological status index ( mmse ) versus rpd patients ( n = 27 ) were analyzed . the...","anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression . although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd . anxiety has been shown to negatively affect cognition in pd . however , our understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind tha...","research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and ..."
1,"small non - coding rnas are transcribed into mrna but remain untranslated in eukaryotic cells . they include sirna ( small interfering rna ) , mirna ( microrna ) , pirna ( piwi - interacting rna ) and snorna ( small nucleolar rna ) . mirnas are a class of multifunctional singled - stranded small rna which are ~20 nt in length and regulate the stability or translational efficiency of targeted messenger rna depending on the base - pairing complementarity between the mirna and its target mrna [ 1 , 2 ] . although over 1,000 mirna sequences have been identified from the tissues or cells of ...","backgroundsmall non - coding rnas are transcribed into mrna but remain untranslated in eukaryotic cells . \n they include sirna ( small interfering rna ) , mirna ( microrna ) , pirna ( piwi - interacting rnas ) and snorna ( small nucleolar rna ). \n mirnas are a class of multifunctional singled - stranded smal

### Calculation of Metrics

In [ ]:
# Installing packages to be used to calculate metrics
!pip install nltk rouge-score bert-score sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
from rouge_score import rouge_scorer
import bert_score
import sacrebleu

# A function which calculates all the metrics
def evaluate_summary(predicted_summary, ground_truth_summary):
    # ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(ground_truth_summary, predicted_summary)
    rouge_1 = scores['rouge1'].fmeasure
    rouge_2 = scores['rouge2'].fmeasure
    rouge_l = scores['rougeL'].fmeasure

    # BERTScore
    bert_precision, bert_recall, bert_f1 = bert_score.score([predicted_summary], [ground_truth_summary], lang='en', model_type='bert-base-uncased')

    # BLEU score
    predicted_tokens = nltk.word_tokenize(predicted_summary.lower())
    ground_truth_tokens = nltk.word_tokenize(ground_truth_summary.lower())
    bleu_score = nltk.translate.bleu_score.sentence_bleu([ground_truth_tokens], predicted_tokens)

    # SacreBLEU
    sacrebleu_score = sacrebleu.corpus_bleu([predicted_summary], [[ground_truth_summary]])

    # METEOR
    meteor_score = nltk.translate.meteor_score.meteor_score([ground_truth_tokens], predicted_tokens)

    return {
        'rouge-1': rouge_1,
        'rouge-2': rouge_2,
        'rouge-l': rouge_l,
        'bertscore-precision': bert_precision.item(), 
        'bertscore-recall': bert_recall.item(), 
        'bertscore-f1': bert_f1.item(),
        'bleu': bleu_score,
        'sacrebleu': sacrebleu_score.score,
        'meteor': meteor_score
    }

In [ ]:
import numpy as np

# A function which calls the evaluare summary function, for list of summaries and ground truth
def get_eval_metrics(candidate_sum, reference_sum):
  rouge1_scores = []
  rouge2_scores = []
  rougeL_scores = []
  bert_score_p = []
  bert_score_r = []
  bert_score_f1 = []
  bleu_scores = []
  sacrebleu_scores = []
  meteor_scores = []

  for i in range(len(candidate_sum)):
    metrics = evaluate_summary(candidate_sum[i], reference_sum[i])

    rouge1_scores.append(metrics['rouge-1'])
    rouge2_scores.append(metrics['rouge-2'])
    rougeL_scores.append(metrics['rouge-l'])

    bert_score_p.append(metrics['bertscore-precision'])
    bert_score_r.append(metrics['bertscore-recall'])
    bert_score_f1.append(metrics['bertscore-f1'])

    bleu_scores.append(metrics['bleu'])
    sacrebleu_scores.append(metrics['sacrebleu'])
    meteor_scores.append(metrics['meteor'])

  return {
      'rouge-1': np.mean(rouge1_scores),
      'rouge-2': np.mean(rouge2_scores),
      'rouge-l': np.mean(rougeL_scores),
      'bertscore-precision': np.mean(bert_score_p), 
      'bertscore-recall': np.mean(bert_score_r), 
      'bertscore-f1': np.mean(bert_score_f1),
      'bleu': np.mean(bleu_scores),
      'sacrebleu': np.mean(sacrebleu_scores),
      'meteor': np.mean(meteor_scores)
  }

<b>For BART</b>

In [ ]:
predicted_summary = results_BartT5_Pegasus_df["Bart_Gen_Summary"]
ground_truth_summary = results_BartT5_Pegasus_df["reference_summary"]
bart_scores = get_eval_metrics(predicted_summary, ground_truth_summary)

In [ ]:
# Results for BART Score
bart_scores

{'rouge-1': 0.37360333333395135,
 'rouge-2': 0.16042688578105405,
 'rouge-l': 0.2571635897245544,
 'bertscore-precision': 0.6521112106063149,
 'bertscore-recall': 0.6046236672184684,
 'bertscore-f1': 0.6253015724095431,
 'bleu': 0.07677426683661569,
 'sacrebleu': 8.704352468679211,
 'meteor': 0.23721571628062071}

<b>For T5</b>

In [ ]:
predicted_summary = results_BartT5_Pegasus_df["T5_Gen_Summary"]
ground_truth_summary = results_BartT5_Pegasus_df["reference_summary"]
t5_scores = get_eval_metrics(predicted_summary, ground_truth_summary)

In [ ]:
# Results for T5
t5_scores

{'rouge-1': 0.4008208242251887,
 'rouge-2': 0.13504248666439614,
 'rouge-l': 0.24264236246232793,
 'bertscore-precision': 0.6313560225746848,
 'bertscore-recall': 0.6240223808722063,
 'bertscore-f1': 0.625616582957181,
 'bleu': 0.08507551910837848,
 'sacrebleu': 9.098023358134133,
 'meteor': 0.2684689117250793}

<b>For Pegasus</b>

In [ ]:
predicted_summary = results_BartT5_Pegasus_df["Pegasus_Gen_Summary"]
ground_truth_summary = results_BartT5_Pegasus_df["reference_summary"]
pegasus_scores = get_eval_metrics(predicted_summary, ground_truth_summary)

In [ ]:
# Results for Pegasus
pegasus_scores

{'rouge-1': 0.4402336430643057,
 'rouge-2': 0.17422600135745264,
 'rouge-l': 0.25060231114087017,
 'bertscore-precision': 0.6377523649822582,
 'bertscore-recall': 0.661047258160331,
 'bertscore-f1': 0.6467721895738081,
 'bleu': 0.11860406769339003,
 'sacrebleu': 12.78788529124314,
 'meteor': 0.34520792629647584}

<b>For GPT3</b>

In [ ]:
predicted_summary = results_df["GPT_summary"]
ground_truth_summary = results_df["reference_summary"]
gpt_scores = get_eval_metrics(predicted_summary, ground_truth_summary)

In [ ]:
# Results for GPT3
gpt_scores

{'rouge-1': 0.45729488745243857,
 'rouge-2': 0.17829806155658995,
 'rouge-l': 0.2783718961710552,
 'bertscore-precision': 0.6565241271799261,
 'bertscore-recall': 0.6616951173002069,
 'bertscore-f1': 0.6585256078026511,
 'bleu': 0.1175864113250646,
 'sacrebleu': 8.416554426874193,
 'meteor': 0.31643575605767976}

<b>For GPT4</b>

In [ ]:
gpt4_df = pd.read_csv("GPT4_responses.csv")

In [ ]:
predicted_summary = gpt4_df["GPT4_Summary"]
ground_truth_summary = results_df["reference_summary"]
gpt4_scores = get_eval_metrics(predicted_summary, ground_truth_summary)

In [ ]:
# Results for GPT4
gpt4_scores

{'rouge-1': 0.49864013597092177,
 'rouge-2': 0.21032328131161238,
 'rouge-l': 0.31881340658874885,
 'bertscore-precision': 0.6682558276436545,
 'bertscore-recall': 0.6995635032653809,
 'bertscore-f1': 0.6832759326154535,
 'bleu': 0.12367896937325663,
 'sacrebleu': 10.5312713027105,
 'meteor': 0.3715520311666836}

## Creating Final DataFrame for Results

In [ ]:
results_BartT5_Pegasus_df["GPT_Gen_Summary"] = results_df["GPT_summary"]

In [ ]:
results_BartT5_Pegasus_df.to_csv("LLMs_testResults.csv")

In [ ]:
bart_res_df = pd.DataFrame(bart_scores, index=[0])
t5_res_df = pd.DataFrame(t5_scores, index=[0])
pegasus_res_df = pd.DataFrame(pegasus_scores, index=[0])
gpt3_5_res_df = pd.DataFrame(gpt_scores, index=[0])
gpt4_res_df = pd.DataFrame(gpt4_scores, index=[0])

resDf = pd.concat([bart_res_df, t5_res_df, pegasus_res_df, gpt3_5_res_df, gpt4_res_df], ignore_index=True)

In [ ]:
name = ['BART', 'T5', "Pegasus", "GPT3", "GPT4"]
resDf['Model-Name'] = name

In [ ]:
resDf2 = resDf[['Model-Name'] + np.delete(resDf.columns.values, np.where(resDf.columns.values == 'Model-Name')).tolist()]

In [ ]:
resDf2

,Model-Name,rouge-1,rouge-2,rouge-l,bertscore-precision,bertscore-recall,bertscore-f1,bleu,sacrebleu,meteor
0,BART,0.373603,0.160427,0.257164,0.652111,0.604624,0.625302,0.076774,8.704352,0.237216
1,T5,0.400821,0.135042,0.242642,0.631356,0.624022,0.625617,0.085076,9.098023,0.268469
2,Pegasus,0.440234,0.174226,0.250602,0.637752,0.661047,0.646772,0.118604,12.787885,0.345208
3,GPT3.5,0.457295,0.178298,0.278372,0.656524,0.661695,0.658526,0.117586,8.416554,0.316436
4,GPT4,0.498640,0.210323,0.318813,0.668256,0.699564,0.683276,0.123679,10.531271,0.371552


In [ ]:
resDf2.to_csv('Final_Results.csv')